In [1]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
import torch
from UITws_v1 import UITws_v1

c:\Users\ACER\anaconda3\envs\env1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class FakenewsClassifier(nn.Module):
    def __init__(self, n_classes, dropout_rate=0.3):
        super(FakenewsClassifier, self).__init__()
        # Khởi tạo phoBERT
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")

        # Classifier network
        self.d1 = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 64)
        self.bn1 = nn.LayerNorm(64)
        self.d2 = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(64, n_classes)
        
        # Khởi tạo trọng số theo normal
        nn.init.normal_(self.fc1.weight, std=0.02)
        nn.init.normal_(self.fc1.bias, 0)
        nn.init.normal_(self.fc2.weight, std=0.02)
        nn.init.normal_(self.fc2.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False,
        )

        x = self.d1(output)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.d2(x)
        x = self.fc2(x)
        return x

In [3]:
# Hàm load mô hình
def load_model():
    # Xác định đường dẫn
    model_path = "./checkpoints/PhoBERT_1024.pth"
    
    # Xác định thiết bị tính toán
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    # Khởi tạo instance model
    model = FakenewsClassifier(n_classes=3, dropout_rate=0.3)
    
    # Chuyển model vào thiết bị tính toán
    model = model.to(device)
    
    # Load model
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    return model

In [4]:
# Hàm load tokenizer
def load_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
    return tokenizer

In [5]:
# Hàm segmentation words
def segmentation_words(text: str):
    uitws_v1 = UITws_v1('./checkpoints/base_sep.pkl')
    return uitws_v1.segment(texts=[text],
                            pre_tokenized=True,
                            batch_size=256)[0]

In [6]:
# Hàm chuyển dữ liệu đầu vào thành dữ liệu mô hình có thể hiểu được
def convert_data(text:str, tokenizer: AutoTokenizer):
    # Xác định các thông số
    fix_len = 1024
    max_len = 256
    stride = 255
    
    # Segmentation words
    text = segmentation_words(text)
    
    # Chứa các sub-tokenizer của content
    encoding_texts = []
    
    # Bắt đầu thực hiện các sub-token
    start = 0
    
    # Hiển thị độ dài của câu
    print(f'### Len of text: {len(text.split())} ###')
    while start < fix_len:
        end = min(start + max_len, fix_len)
            
        # Lấy phân đoạn của văn bản từ `start` đến `end`
        segment = " ".join(text.split()[start:end])
        
        print(f'=>start: {start}-end: {end}')
        print(f'=>segment: {segment}')
        
        # Tiến hành tokenizer
        encoding_text = tokenizer.encode_plus(
        segment,
        truncation=True,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
        )
        
        # Thêm vào danh sách chứa các sub-tokenizer của content
        encoding_texts.append(encoding_text)
        
        print("-----------")
        
        # Di chuyển chỉ số bắt đầu dựa trên stride
        start += stride
        
        # Kiểm tra điều kiện
        # if start >= len(text.split()):
        #     break
        
    # Flatten encoding context
    for encoding_text in encoding_texts:
        encoding_text['input_ids'] = encoding_text['input_ids'].flatten()
        encoding_text['attention_mask'] = encoding_text['attention_mask'].flatten()
    
        
    # Trả về kết quả
    return {
        'encoding_texts': encoding_texts, # danh sách (input_ids, attention_mask)
    }

In [7]:
# Hàm dự đoán kết quả
def predict(text: str, model:FakenewsClassifier, tokenizer: AutoTokenizer):
    # Convert data
    encoding_texts = convert_data(text, tokenizer)['encoding_texts']
    print(f'### Len encoding texts: {len(encoding_texts)} ###')
    
    # Xác định thiết bị tính toán
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    # Biến lưu trữ tổng của outputs_text
    total_outputs_text = None
    num_sub_texts = 0
    
    # Duyệt qua từng sub-text trong texts
    for idx, encoding_text in enumerate(encoding_texts):
        print(f'#idx: {idx}')
        
        encoding_text_input_ids = encoding_text['input_ids'].to(device).unsqueeze(0)
        encoding_text_attention_masks = encoding_text['attention_mask'].to(device).unsqueeze(0)
             
        print(f'=>Encoding_text_input_ids: {encoding_text_input_ids.shape}')
        print(f'=>Encoding_text_attention_masks: {encoding_text_attention_masks.shape}')
        
        # Kiểm tra nếu input_ids toàn là padding token và attention_mask chủ yếu là 0
        if torch.sum(encoding_text_attention_masks) <= 2:
            print(f"Bỏ qua sub-text tại vị trí {idx} vì input_ids toàn padding và attention_mask chủ yếu là 0")
            continue  # Bỏ qua sub-text này
        
        # Không đạo hàm
        with torch.no_grad():
            # Dự đoán cho sub-text
            outputs_text = model(encoding_text_input_ids, attention_mask=encoding_text_attention_masks)
            print(f'Dự đoán thành công')
            
            # Cộng dồn kết quả đầu ra
            if total_outputs_text is None:
                total_outputs_text = outputs_text
            else:
                total_outputs_text += outputs_text
                
            num_sub_texts += 1
        
    # Tính trung bình cộng của outputs_text
    average_outputs_text = total_outputs_text / num_sub_texts

    # Áp dụng softmax để chuyển logits thành xác suất
    probabilities_softmax = torch.nn.functional.softmax(average_outputs_text, dim=-1)
    
    print(f'average_outputs_text: {probabilities_softmax}, shape: {probabilities_softmax.shape}')
    
    
    # Trả về kết quả
    probabilities = {
    "Thật": probabilities_softmax[0][0].item(),
    "Giả do con người": probabilities_softmax[0][1].item(),
    "Giả do AI": probabilities_softmax[0][2].item()
    }
        
    return probabilities

In [8]:
# Khởi tạo mô hình
model = load_model()

In [9]:
# Khởi tạo tokenizer
tokenizer = load_tokenizer()

In [50]:
# Tạo dữ liệu mẫu
content = "23 9, 2010 kính gửi thủ_tướng nguyễn dũng cộng_hoà xã_hội chủ nghiã việt nam đại_sứ việt nam 1233 20th street, nw, suite 400 washington, dc 20036 thưa thủ tướng, mấy tuần cầm_quyền việt nam bắt giam vận_động dân chủ. giáo_sư_phạm minh hoàng, mục_sư dương kim khải, trần thị thuý nguyễn thành_tâm bắt giam đòi thay_đổi dân_chủ ôn_hoà thực_thi_nhân_quyền tham_gia hoạt_động trị tư thành_viên tổ_chức tranh_đấu dân_chủ đảng việt tân. giam lý nêu phi_lý thể chấp được. việt nam ký tuyên_ngôn quốc_tế nhân_quyền cầm_quyền việt nam tiếp_tục bắt sách_nhiễu trí thức, vị lãnh_đạo tôn_giáo_dân_chủ ôn_hoà quan tâm. dân_biểu quốc_hội hỗ_trợ rộng quan_hệ song_phương quyền tiếp_tục bóp_nghẹt thô_bạo quyền bày_tỏ bất_đồng kiến ôn_hoà dân nói, báo chí, mạng internet. tham_gia hoạt_động tranh_đấu ôn_hoà tội hình sự, quyền việt nam gán nhãn_hiệu bất hợp_pháp phi lý. cầm_quyền việt nam tiếp_tục đàn_áp dân quan tâm. bắt giam giáo_sư_phạm minh hoàng phản_đối khai_thác quặng mỏ bô-xít tây_nguyên viết đăng trang blog vô lý. quyền bạo_lực đối nguyễn thành_tâm trần thị thuý giúp nông_dân đòi công_xã_hội thể chấp nhận. kinh_hãi việt nam mục_sư dương kim khải tội_nhân ta thực_thi quyền tôn_giáo áp_dụng răn dậy tôn_giáo sống hàng giúp_đỡ nông_dân cướp ruộng_đất khiếu nại. cầm_quyền việt nam tiếp_tục biến quyền ngôn luận, quyền phát biểu, quyền hội_họp quyền tôn_giáo thành tội hình_thể chấp được. thể tiếp_tục thành_viên thực cộng_đồng giới, cầm_quyền việt nam ngưng sách_nhiễu đàn_áp dân phát_biểu ôn_hoà tham_gia hoạt_động trị. dân_biểu quốc_hội quan_tâm dạn thách_đố cầm_quyền việt nam phép tất công_dân_quyền phát_biểu phép tham_gia sinh_hoạt đấu tranh. kêu_gọi cầm_quyền việt nam lập_tức ngừng sách_nhiễu dân thả tù_nhân trị hiện giam phản_đối quyết_định cầm quyền. trân trọng, loretta sanchez dân_biểu quốc_hội joseph dân_biểu quốc_hội zoe lofgren dân_biểu quốc_hội dan burton dân_biểu quốc_hội gerald connolly dân_biểu quốc_hội daniel lungren dân_biểu quốc_hội david wu dân_biểu quốc_hội john culbertson dân_biểu quốc_hội judy chu dân_biểu quốc_hội james mcgovern dân_biểu quốc hội. cc: trưởng ngoại_giao hillary rodham clinton giới_chức việt nam tuyến đường_sắt tốc nối hà nội sài gòn tầm quan_trọng chiến_lược dự_đoán lợi_ích to kinh_tế xã hội, kiến băn_khoăn chi phí, khả_thi mặt thương_mại tác_động hạn_chế kinh tế. tù nghìn_thu ngoài. đi tù nghĩa câu ấy. nhà, hàng trăm em, bè ta mòn_mỏi tù. mong tiếp_tục đồng_hành giúp_đỡ tù_nhân lương_tâm gia_đình chặng đường khắc_nghiệt chông gai. đừng rơi họ, đừng rơi nào. quan_sát câu mông cổ lãnh_đạo việt nam thăm thăm hoa kỳ, cuba. quan_sát giải mặt địa chiến_lược hai đường trái_ngược việt nam mông cổ liên xổ sụp đổ. vui ấm phái_đoàn hoà lan, bỉ, đức, thuỵ sĩ, mỹ quản_ngại đường xá xôi cộng_đồng việt pháp tiếng phản_đối án tô lâm, tổng bí_thư đảng csvn kiêm chủ_tịch nước, tội_nhân đàn_áp dã_man vụ án đồng_tâm bắt dân chủ, tiếng ôn hoà, đấu_tranh việt nam chế xã_hội nhân_công hơn, xây_dựng tảng do, dân_chủ_nhân quyền. việt tân tập_hợp việt yêu dân_chủ khát_vọng_canh_tân canh_tân việt nam hoạt_động đấu_tranh bất bạo động. "
# Dự đoán
probabilities = predict(content, model, tokenizer)

c:\Users\ACER\anaconda3\envs\env1\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Len of text: 478 ###
=>start: 0-end: 256
=>segment: 23 9, 2010 kính gửi thủ_tướng nguyễn dũng cộng_hoà xã_hội chủ nghiã việt nam đại_sứ việt nam 1233 20th street, nw, suite 400 washington, dc 20036 thưa thủ tướng, mấy tuần cầm_quyền việt nam bắt giam vận_động dân chủ. giáo_sư_phạm minh hoàng, mục_sư dương kim khải, trần thị thuý nguyễn thành_tâm bắt giam đòi thay_đổi dân_chủ ôn_hoà thực_thi_nhân_quyền tham_gia hoạt_động trị tư thành_viên tổ_chức tranh_đấu dân_chủ đảng việt tân. giam lý nêu phi_lý thể chấp được. việt nam ký tuyên_ngôn quốc_tế nhân_quyền cầm_quyền việt nam tiếp_tục bắt sách_nhiễu trí thức, vị lãnh_đạo tôn_giáo_dân_chủ ôn_hoà quan tâm. dân_biểu quốc_hội hỗ_trợ rộng quan_hệ song_phương quyền tiếp_tục bóp_nghẹt thô_bạo quyền bày_tỏ bất_đồng kiến ôn_hoà dân nói, báo chí, mạng internet. tham_gia hoạt_động tranh_đấu ôn_hoà tội hình sự, quyền việt nam gán nhãn_hiệu bất hợp_pháp phi lý. cầm_quyền việt nam tiếp_tục đàn_áp dân quan tâm. bắt giam giáo_sư_phạm minh hoàng phản_đố

In [51]:
probabilities

{'Thật': 0.03266631066799164,
 'Giả do con người': 0.9527502655982971,
 'Giả do AI': 0.014583307318389416}

In [2]:
# Hàm remove stopword
def remove_stopwords(text:str):
    # Load stopword
    stop_words = []
    with open('./checkpoints/vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
        stop_words = f.read().splitlines()

    words = text.split()
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [3]:
text = """Nguồn nhân lực chất lượng cao được tập trung phát triển, gắn với khoa học công nghệ, đổi mới sáng tạo, khởi nghiệp. Năm 2024, Việt Nam đứng thứ 3 ASEAN về đầu tư vào Start-up; thứ 56 thế giới, thứ 5 khu vực Đông Nam Á và thứ 12 khu vực châu Á - Thái Bình Dương về Chỉ số hệ sinh thái khởi nghiệp toàn cầu.

Tỷ lệ thất nghiệp khu vực thành thị ước cả năm dưới 4%. Chính phủ cũng dành gần 700.000 tỷ đồng để tăng mức lương cơ sở lên 30% cho cán bộ, công chức, viên chức, lực lượng vũ trang. Lương hưu, bảo hiểm xã hội, trợ cấp ưu đãi người có công và trợ cấp xã hội từ ngày 1/7 được tăng với mức cao nhất từ trước đến nay.

Tuy nhiên, Thủ tướng nhìn nhận kinh tế vĩ mô còn tiềm ẩn rủi ro từ lạm phát, tỷ giá. Tín dụng tăng trưởng chưa cao; áp lực trả nợ trái phiếu doanh nghiệp đến hạn lớn. Hoạt động sản xuất, kinh doanh gặp nhiều khó khăn. Sức mua trong nước có dấu hiệu tăng chậm lại.

Thể chế, pháp luật còn một số vướng mắc. Việc phân cấp, phân quyền, cắt giảm một số quy định, thủ tục hành chính còn rườm rà, chưa triệt để. Chất lượng nguồn nhân lực, nhất là ngành công nghệ cao chưa đáp ứng yêu cầu. Đời sống của một bộ phận người dân còn khó khăn. Thiên tai, bão lũ diễn biến khó lường, nhất là cơn bão Yagi gây thiệt hại nghiêm trọng.

Phương hướng những tháng cuối năm, Chính phủ chú trọng ưu tiên thúc đẩy tăng trưởng gắn với giữ vững ổn định kinh tế vĩ mô, kiểm soát lạm phát và bảo đảm các cân đối lớn của nền kinh tế, an ninh năng lượng, an ninh lương thực. Trong đó, tốc độ tăng GDP phấn đấu đạt khoảng 7%."""

In [2]:
bert = AutoModel.from_pretrained("vinai/phobert-base")

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(bert)

134998272

In [3]:
bert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou